In [ ]:
# import the necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, Conv2DTranspose, Lambda
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt

# load the datasets
img_dir = r'C:\Users\user\Desktop\abdominal data\abdominal_US\abdominal_US\AUS\images\train'
mask_dir = r'C:\Users\user\Desktop\abdominal data\abdominal_US\abdominal_US\AUS\annotations\train'



# Load and preprocess the training images.
img_names = os.listdir(img_dir)
imgs = [Image.open(os.path.join(img_dir, img_name)) for img_name in img_names]
X = np.array([np.array(img.resize((464, 464))) for img in imgs])
X = X / 255.0  # Normalize the images to [0, 1] range.

# Define a mapping from RGB values to class labels.
colors_to_labels = {
    (255, 255, 0): 0,  # Yellow
    (255, 0, 255): 1,  # Red
    (100, 0, 100): 2,  # Darker red
    (0, 255, 0): 3  # Green
}

# Load, preprocess, and encode the masks.
mask_names = os.listdir(mask_dir)
masks = [Image.open(os.path.join(mask_dir, mask_name)) for mask_name in mask_names]
masks = [mask.resize((464, 464)) for mask in masks] 
Y = np.empty((len(masks), 464, 464, len(colors_to_labels)))

for i, mask in enumerate(masks):
    mask_array = np.array(mask)
    for j, (color, label) in enumerate(colors_to_labels.items()):
        Y[i, :, :, j] = np.all(mask_array == np.array(color).reshape(1, 1, 3), axis=2)

# Split the data into a training set and a validation set.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the U-Net model.
inputs = Input((464, 464, 3))
conv1 = Conv2D(64, 4, activation='relu', padding='same')(inputs)
conv1 = Conv2D(64, 4, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(128, 4, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, 4, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(256, 4, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, 4, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(512, 4, activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, 4, activation='relu', padding='same')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
conv5 = Conv2D(1024, 4, activation='relu', padding='same')(pool4)
conv5 = Conv2D(1024, 4, activation='relu', padding='same')(conv5)
drop5 = Dropout(0.5)(conv5)
up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
merge6 = concatenate([drop4, up6], axis=3)
conv6 = Conv2D(512, 4, activation='relu', padding='same')(merge6)
conv6 = Conv2D(512, 4, activation='relu', padding='same')(conv6)
up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
merge7 = concatenate([conv3, up7], axis=3)
conv7 = Conv2D(256, 4, activation='relu', padding='same')(merge7)
conv7 = Conv2D(256, 4, activation='relu', padding='same')(conv7)
up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
merge8 = concatenate([conv2, up8], axis=3)
conv8 = Conv2D(128, 4, activation='relu', padding='same')(merge8)
conv8 = Conv2D(128, 4, activation='relu', padding='same')(conv8)
up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
merge9 = concatenate([conv1, up9], axis=3)
conv9 = Conv2D(64, 4, activation='relu', padding='same')(merge9)
conv9 = Conv2D(64, 4, activation='relu', padding='same')(conv9)
conv9 = Conv2D(64,4, activation='relu', padding='same')(conv9)
conv10 = Conv2D(4, 1, activation='softmax')(conv9)

model = Model(inputs, conv10)

model.compile(optimizer = Adam(learning_rate = 1e-4), loss = 'categorical_crossentropy')

# Fit the model to the training data.
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=4, epochs=10)


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

img_dir = r'C:\Users\user\Desktop\New folder (3)\abdominal_US\abdominal_US\AUS\images\train'
mask_dir = r'C:\Users\user\Desktop\New folder (3)\abdominal_US\abdominal_US\AUS\annotations\train'

image_list = [os.path.join(img_dir, img_name) for img_name in os.listdir(img_dir)]
mask_list = [os.path.join(mask_dir, mask_name) for mask_name in os.listdir(mask_dir)]

image_list_ds = tf.data.Dataset.from_tensor_slices(image_list)
mask_list_ds = tf.data.Dataset.from_tensor_slices(mask_list)

# Ensure the lists are in the same order
image_list_ds = tf.data.Dataset.from_tensor_slices(image_list)
mask_list_ds = tf.data.Dataset.from_tensor_slices(mask_list)

# Create a Dataset of (image, mask) pairs
dataset = tf.data.Dataset.zip((image_list_ds, mask_list_ds))

def process_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    
    return img, mask

def preprocess(image, mask):
    input_image = tf.image.resize(image, (464, 464), method='nearest')
    input_mask = tf.image.resize(mask, (464, 464), method='nearest')

    return input_image, input_mask

dataset = dataset.map(process_path)
dataset = dataset.map(preprocess)

# Verify the preprocessing
for image, mask in dataset.take(1):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Image')
    plt.subplot(1, 2, 2)
    plt.imshow(mask)
    plt.title('Mask')
    plt.show()


In [ ]:
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D
#defining the encoding part of our U-net
def conv_block(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):
    """
    Convolutional downsampling block
    
    Arguments:
        inputs -- Input tensor
        n_filters -- Number of filters for the convolutional layers
        dropout_prob -- Dropout probability
        max_pooling -- Use MaxPooling2D to reduce the spatial dimensions of the output volume
    Returns: 
        next_layer, skip_connection --  Next layer and skip connection outputs
    """

    conv = Conv2D(n_filters, 
                  4,     
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(inputs)
    conv = Conv2D(n_filters,
                  4,   
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(conv)
    
    # if dropout_prob > 0 add a dropout layer, with the variable dropout_prob as parameter
    if dropout_prob > 0:
         conv = Dropout(dropout_prob)(conv)
         
    # if max_pooling is True add a MaxPooling2D with 2x2 pool_size
    if max_pooling:
        next_layer = MaxPooling2D(pool_size=(2, 2))(conv)
        
    else:
        next_layer = conv
        
    skip_connection = conv
    
    return next_layer, skip_connection


In [ ]:
#defining our decoding part of our U-net
from tensorflow.keras.layers import Conv2DTranspose, concatenate

def upsampling_block(expansive_input, contractive_input, n_filters=32):
    """
    Convolutional upsampling block
    
    Arguments:
        expansive_input -- Input tensor from previous layer
        contractive_input -- Input tensor from previous skip layer
        n_filters -- Number of filters for the convolutional layers
    Returns: 
        conv -- Tensor output
    """
    
    up = Conv2DTranspose(
                 n_filters,    
                 (4, 4),   
                 strides=(2, 2),
                 padding='same')(expansive_input)
    merge = concatenate([up, contractive_input], axis=3)
    conv = Conv2D(n_filters,  
                 (4, 4),    
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(merge)
    conv = Conv2D(n_filters,  
                 (4, 4),  
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(conv)
    
    return conv


In [ ]:

def unet_model(input_size=(464, 464, 3), n_filters=32, n_classes=23):
    """
    Unet model
    
    Arguments:
        input_size -- Input shape 
        n_filters -- Number of filters for the convolutional layers
        n_classes -- Number of output classes
    Returns: 
        model -- tf.keras.Model
    """
    inputs = Input(input_size)
    
    cblock1 = conv_block(inputs, n_filters)
    cblock2 = conv_block(cblock1[0], n_filters*2)
    cblock3 = conv_block(cblock2[0], n_filters*4)
    cblock4 = conv_block(cblock3[0], n_filters*8, dropout_prob=0.3) 
    cblock5 = conv_block(cblock4[0], n_filters*16, dropout_prob=0.3, max_pooling=False) 
    ublock6 = upsampling_block(cblock5[0], cblock4[1],  n_filters*8)
    ublock7 = upsampling_block(ublock6, cblock3[1],  n_filters*4)
    ublock8 = upsampling_block(ublock7, cblock2[1],  n_filters*2)
    ublock9 = upsampling_block(ublock8, cblock1[1],  n_filters)

    conv9 = Conv2D(n_filters,
                 (3, 3),
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

    conv10 = Conv2D(n_classes, (1, 1), padding='same')(conv9)
    
    model = tf.keras.Model(inputs=inputs, outputs=conv10)

    return model
img_height = 464
img_width = 464
num_channels = 3

unet = unet_model((img_height, img_width, num_channels))

In [ ]:
unet.summary()

In [ ]:
unet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
EPOCHS = 5
BUFFER_SIZE = 500
BATCH_SIZE = 32

# Preparing the dataset for training by caching, shuffling, and batching
train_dataset = dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Compile the model
unet.compile(optimizer=Adam(), 
             loss=SparseCategoricalCrossentropy(from_logits=True), 
             metrics=['accuracy'])

# Train the model
model_history = unet.fit(train_dataset, epochs=EPOCHS)
